# Time-Domain Relaxation Validation

Validate RheoJAX relaxation fits on the PyVisco time-domain dataset.

In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports
Fit relaxation modulus data and compare Prony vs fractional models.

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)


INFO:2025-12-06 04:15:40,493:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load relaxation data

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyvisco' / 'time_master'
relax_df = pd.read_csv(DATA_DIR / 'time_user_master.csv')

relax_clean = relax_df.iloc[1:].astype(float)
t = relax_clean['t'].to_numpy()
E_t = relax_clean['E_relax'].to_numpy()

data = RheoData(x=t, y=E_t, x_units='s', y_units='MPa', domain='relaxation')
print(data)


RheoData(x=array([2.8176e-03, 2.5827e-02, 5.5724e-02, 8.9250e-02, 1.2278e-01,
       1.5215e-01, 1.8103e-01, 2.1337e-01, 2.4577e-01, 2.7918e-01,
       3.1173e-01, 3.4513e-01, 3.7754e-01, 4.1103e-01, 4.4346e-01,
       4.7695e-01, 5.0937e-01, 5.4287e-01, 5.7526e-01, 6.0880e-01,
       6.4121e-01, 6.7472e-01, 7.0711e-01, 7.4063e-01, 9.3044e-01,
       1.1662e+00, 1.4011e+00, 1.6631e+00, 1.9252e+00, 2.1966e+00,
       2.4588e+00, 2.7300e+00, 2.9922e+00, 3.2635e+00, 3.5256e+00,
       3.7965e+00, 4.0590e+00, 4.3273e+00, 4.5926e+00, 4.8585e+00,
       5.1262e+00, 5.3888e+00, 5.6595e+00, 5.9218e+00, 6.1930e+00,
       7.2434e+00, 8.9706e+00, 1.0696e+01, 1.2402e+01, 1.3982e+01,
       1.5550e+01, 1.7197e+01, 1.8912e+01, 2.0593e+01, 2.2271e+01,
       2.3988e+01, 2.5713e+01, 2.7440e+01, 2.9108e+01, 3.0835e+01,
       3.2562e+01, 3.4242e+01, 3.5957e+01, 3.7681e+01, 4.1493e+01,
       5.5348e+01, 6.9218e+01, 8.3025e+01, 9.6886e+01, 1.1076e+02,
       1.2459e+02, 1.3843e+02, 1.5232e+02, 1.6615e+

## Fit models

In [4]:
gm = GeneralizedMaxwell(n_modes=6, modulus_type='tensile')
gm.fit(t, E_t, test_mode='relaxation', use_log_residuals=True, use_multi_start=True)
gm_pred = gm.predict(t)
gm_r2 = gm.score(t, E_t)

fm = FractionalMaxwellModel()
fm.fit(t, E_t, test_mode='relaxation', use_log_residuals=True)
fm_pred = fm.predict(t, test_mode='relaxation')
fm_r2 = r2_complex(E_t, fm_pred)


print(f"Generalized Maxwell R^2: {gm_r2:.4f}")
print(f"Fractional Maxwell R^2:   {fm_r2:.4f}")


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.326410s


Convergence: reason=`xtol` termination condition is satisfied. | iterations=13 | final_cost=4.790065e+07 | time=1.326s | final_gradient_norm=1566957452.1387854


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.365283s


...

Inner optimization loop hit iteration limit | {'inner_iterations': 100, 'actual_reduction': -1}


Timer: optimization took 1.362439s


Convergence: reason=Inner optimization loop exceeded maximum iterations. | iterations=1 | final_cost=6.616982e+05 | time=1.362s | final_gradient_norm=nan


NLSQ hit inner iteration limit; retrying with SciPy least_squares for stability.


Generalized Maxwell R^2: 0.4344
Fractional Maxwell R^2:   -22784.2528


## Plot relaxation fits

In [5]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(t, E_t, 'o', label='Data', alpha=0.5)
ax.loglog(t, gm_pred, '-', label=f'Generalized Maxwell (R^2={gm_r2:.3f})')
ax.loglog(t, fm_pred, '--', label=f'Fractional Maxwell (R^2={fm_r2:.3f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Relaxation modulus (MPa)')
ax.grid(True, which='both', ls='--', alpha=0.4)
ax.legend()
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_14349/3862213224.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
